In [1]:
pip install timm

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import timm

In [3]:
# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

# Configuration
class Config:
    # Paths
    DATA_ROOT = "/kaggle/input/croppedcelebdfv2"
    FAKE_DIR = os.path.join(DATA_ROOT, "fake")
    REAL_DIR = os.path.join(DATA_ROOT, "real")
    OUTPUT_DIR = "/kaggle/working"
    
    # Model parameters
    INPUT_SIZE = 299  # Xception input size
    BATCH_SIZE = 32
    NUM_WORKERS = 4
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Training parameters
    EPOCHS_CLASSIFIER = 5  # Train only classifier
    EPOCHS_FINETUNE = 10   # Fine-tune entire model
    LEARNING_RATE_CLASSIFIER = 5e-4
    LEARNING_RATE_FINETUNE = 5e-5
    WEIGHT_DECAY = 1e-5
    
    # Checkpointing
    CHECKPOINT_FREQ = 1  # Save model every N epochs
    
    # Video aggregation
    AGGREGATION_METHOD = "mean"  # Options: "mean", "max", "voting"
    
    # Class mapping
    CLASS_MAP = {"fake": 0, "real": 1}


In [4]:
config = Config()
print(f"Using device: {config.DEVICE}")

Using device: cuda


In [5]:
# Data preparation
class DeepfakeDataset(Dataset):
    """Dataset for deepfake detection using face frames from videos"""
    
    def __init__(self, video_paths, labels, transform=None):
        """
        Args:
            video_paths: List of paths to video frame directories
            labels: List of labels for each video (0: fake, 1: real)
            transform: PyTorch transforms for data augmentation
        """
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform
        
        # Create a flattened list of (frame_path, label) pairs
        self.frame_paths = []
        self.frame_labels = []
        self.video_indices = []  # Track which video each frame belongs to
        
        for i, (video_path, label) in enumerate(zip(video_paths, labels)):
            frames = [os.path.join(video_path, f) for f in os.listdir(video_path) 
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            
            self.frame_paths.extend(frames)
            self.frame_labels.extend([label] * len(frames))
            self.video_indices.extend([i] * len(frames))
    
    def __len__(self):
        return len(self.frame_paths)
    
    def __getitem__(self, idx):
        frame_path = self.frame_paths[idx]
        label = self.frame_labels[idx]
        
        # Load image
        image = Image.open(frame_path).convert('RGB')
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        
        return image, label, self.video_indices[idx]

In [6]:
def get_transforms(train=True):
    """
    Get transformations for training or validation
    
    Args:
        train: Whether to include training augmentations
    
    Returns:
        Composed transformations
    """
    if train:
        return transforms.Compose([
            transforms.Resize((config.INPUT_SIZE, config.INPUT_SIZE)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomAffine(degrees=5, translate=(0.05, 0.05), scale=(0.95, 1.05)),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize((config.INPUT_SIZE, config.INPUT_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

def prepare_data():
    """
    Prepare training and validation datasets and dataloaders
    
    Returns:
        Train and validation dataloaders
    """
    # Get all video directories and their labels
    fake_videos = [os.path.join(config.FAKE_DIR, d) for d in os.listdir(config.FAKE_DIR) if os.path.isdir(os.path.join(config.FAKE_DIR, d))]
    real_videos = [os.path.join(config.REAL_DIR, d) for d in os.listdir(config.REAL_DIR) if os.path.isdir(os.path.join(config.REAL_DIR, d))]
    
    all_videos = fake_videos + real_videos
    all_labels = [config.CLASS_MAP["fake"]] * len(fake_videos) + [config.CLASS_MAP["real"]] * len(real_videos)
    
    # Split into train and validation sets
    train_videos, val_videos, train_labels, val_labels = train_test_split(
        all_videos, all_labels, test_size=0.2, random_state=42, stratify=all_labels
    )
    
    print(f"Train videos: {len(train_videos)} | Validation videos: {len(val_videos)}")
    
    # Create datasets
    train_dataset = DeepfakeDataset(
        train_videos,
        train_labels,
        transform=get_transforms(train=True)
    )
    
    val_dataset = DeepfakeDataset(
        val_videos,
        val_labels,
        transform=get_transforms(train=False)
    )
    
    print(f"Train frames: {len(train_dataset)} | Validation frames: {len(val_dataset)}")
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=True,
        num_workers=config.NUM_WORKERS,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.BATCH_SIZE,
        shuffle=False,
        num_workers=config.NUM_WORKERS,
        pin_memory=True
    )
    
    return train_loader, val_loader, train_dataset, val_dataset


In [7]:
# Prepare data
train_loader, val_loader, train_dataset, val_dataset = prepare_data()

Train videos: 4983 | Validation videos: 1246
Train frames: 64994 | Validation frames: 16342


In [8]:
# Model definition
class XceptionDeepfakeDetector(nn.Module):
    """Xception-based deepfake detection model"""
    
    def __init__(self, num_classes=2, pretrained=True):
        super(XceptionDeepfakeDetector, self).__init__()
        
        # Load pretrained Xception model
        #self.xception = models.xception(pretrained=pretrained)
        self.xception = timm.create_model('xception', pretrained=True)
        
        # Remove the original classifier
        in_features = self.xception.fc.in_features
        self.xception.fc = nn.Identity()
        
        # Add custom classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        # Extract features with Xception
        features = self.xception(x)
        
        # Apply classifier
        output = self.classifier(features)
        
        return output
    
    def freeze_backbone(self):
        """Freeze the Xception backbone"""
        for param in self.xception.parameters():
            param.requires_grad = False
    
    def unfreeze_backbone(self):
        """Unfreeze the Xception backbone for fine-tuning"""
        for param in self.xception.parameters():
            param.requires_grad = True

In [9]:
# Training functions
def train_epoch(model, dataloader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(dataloader, desc="Training")
    for inputs, targets, _ in pbar:
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
        # Update progress bar
        pbar.set_postfix({"loss": f"{running_loss/total:.4f}", "acc": f"{100 * correct/total:.2f}%"})
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

def validate(model, dataloader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    all_preds = []
    all_targets = []
    all_video_indices = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Validating")
        for inputs, targets, video_indices in pbar:
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
            # Store predictions and targets for video-level aggregation
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_video_indices.extend(video_indices.numpy())
            
            # Update progress bar
            pbar.set_postfix({"loss": f"{running_loss/total:.4f}", "acc": f"{100 * correct/total:.2f}%"})
    
    # Calculate metrics for frame-level predictions
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    # Video-level aggregation and evaluation
    video_preds, video_targets = aggregate_video_predictions(all_preds, all_targets, all_video_indices)
    video_acc = accuracy_score(video_targets, video_preds)
    
    return epoch_loss, epoch_acc, video_acc, video_preds, video_targets

In [10]:
def aggregate_video_predictions(frame_preds, frame_targets, video_indices):
    """
    Aggregate frame-level predictions to video-level predictions
    
    Args:
        frame_preds: Frame-level predictions
        frame_targets: Frame-level targets
        video_indices: Indices indicating which video each frame belongs to
    
    Returns:
        Video-level predictions and targets
    """
    # Convert to numpy arrays
    frame_preds = np.array(frame_preds)
    frame_targets = np.array(frame_targets)
    video_indices = np.array(video_indices)
    
    # Get unique video indices
    unique_videos = np.unique(video_indices)
    
    video_preds = []
    video_targets = []
    
    for vid_idx in unique_videos:
        # Get all frames for this video
        mask = video_indices == vid_idx
        vid_preds = frame_preds[mask]
        vid_targets = frame_targets[mask]
        
        # All targets for a video should be the same
        vid_target = vid_targets[0]
        
        # Aggregate frame predictions based on the specified method
        if config.AGGREGATION_METHOD == "mean":
            # Average predictions and round
            vid_pred = int(np.mean(vid_preds) >= 0.5)
        elif config.AGGREGATION_METHOD == "max":
            # If any frame is predicted as fake (0), the video is fake
            vid_pred = 1 if np.all(vid_preds == 1) else 0
        elif config.AGGREGATION_METHOD == "voting":
            # Majority voting
            fake_count = np.sum(vid_preds == 0)
            real_count = np.sum(vid_preds == 1)
            vid_pred = 1 if real_count > fake_count else 0
        else:
            raise ValueError(f"Unknown aggregation method: {config.AGGREGATION_METHOD}")
        
        video_preds.append(vid_pred)
        video_targets.append(vid_target)
    
    return np.array(video_preds), np.array(video_targets)

def save_checkpoint(model, optimizer, epoch, loss, acc, stage, save_dir):
    """
    Save model checkpoint
    
    Args:
        model: Model to save
        optimizer: Optimizer state
        epoch: Current epoch
        loss: Validation loss
        acc: Validation accuracy
        stage: Training stage ('classifier' or 'finetune')
        save_dir: Directory to save checkpoint
    """
    os.makedirs(save_dir, exist_ok=True)
    
    checkpoint_path = os.path.join(save_dir, f"deepfake_detector_{stage}_epoch_{epoch}.pth")
    
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        'acc': acc
    }
    
    torch.save(checkpoint, checkpoint_path)
    print(f"Checkpoint saved at {checkpoint_path}")

In [11]:
def calculate_metrics(y_true, y_pred):
    """
    Calculate classification metrics
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        
    Returns:
        Dictionary of metrics
    """
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='weighted'),
        'recall': recall_score(y_true, y_pred, average='weighted'),
        'f1': f1_score(y_true, y_pred, average='weighted'),
    }

def plot_metrics(train_losses, val_losses, train_accs, val_accs, video_accs, stage, save_dir):
    """
    Plot training and validation metrics
    
    Args:
        train_losses: List of training losses
        val_losses: List of validation losses
        train_accs: List of training accuracies
        val_accs: List of validation accuracies
        video_accs: List of video-level validation accuracies
        stage: Training stage ('classifier' or 'finetune')
        save_dir: Directory to save plots
    """
    os.makedirs(save_dir, exist_ok=True)
    
    epochs = range(1, len(train_losses) + 1)
    
    plt.figure(figsize=(12, 5))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
    plt.title(f'{stage} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accs, 'b-', label='Training Accuracy')
    plt.plot(epochs, val_accs, 'r-', label='Validation Frame Accuracy')
    plt.plot(epochs, video_accs, 'g-', label='Validation Video Accuracy')
    plt.title(f'{stage} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f"metrics_{stage}.png"))
    plt.close()
    
    # Save metrics to CSV
    metrics_df = pd.DataFrame({
        'epoch': epochs,
        'train_loss': train_losses,
        'val_loss': val_losses,
        'train_acc': train_accs,
        'val_frame_acc': val_accs,
        'val_video_acc': video_accs
    })
    metrics_df.to_csv(os.path.join(save_dir, f"metrics_{stage}.csv"), index=False)

In [12]:
# Main training function
def train_deepfake_detector():
    # Create model
    model = XceptionDeepfakeDetector(num_classes=2, pretrained=True)
    model = model.to(config.DEVICE)
    
    # Training process
    best_video_acc = 0.0
    
    # STAGE 1: Train only the classifier
    print("\n=== Training Stage 1: Classifier Only ===")
    model.freeze_backbone()
    
    # Verify only classifier parameters are trainable
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable_params} / {total_params} ({trainable_params/total_params:.2%})")
    
    # Define criterion and optimizer for classifier training
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.classifier.parameters(), 
                           lr=config.LEARNING_RATE_CLASSIFIER, 
                           weight_decay=config.WEIGHT_DECAY)
    
    # Training loop
    train_losses_c = []
    val_losses_c = []
    train_accs_c = []
    val_accs_c = []
    video_accs_c = []
    
    for epoch in range(1, config.EPOCHS_CLASSIFIER + 1):
        print(f"\nEpoch {epoch}/{config.EPOCHS_CLASSIFIER}")
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, config.DEVICE)
        
        # Validate
        val_loss, val_acc, video_acc, video_preds, video_targets = validate(model, val_loader, criterion, config.DEVICE)
        
        # Save metrics
        train_losses_c.append(train_loss)
        val_losses_c.append(val_loss)
        train_accs_c.append(train_acc)
        val_accs_c.append(val_acc)
        video_accs_c.append(video_acc)
        
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Frame Acc: {val_acc:.4f} | Val Video Acc: {video_acc:.4f}")
        
        # Save checkpoint if it's the best model so far
        if video_acc > best_video_acc:
            best_video_acc = video_acc
            save_checkpoint(model, optimizer, epoch, val_loss, video_acc, "classifier_best", config.OUTPUT_DIR)
        
        # Save periodic checkpoint
        if epoch % config.CHECKPOINT_FREQ == 0:
            save_checkpoint(model, optimizer, epoch, val_loss, video_acc, f"classifier_epoch{epoch}", config.OUTPUT_DIR)
    
    # # Calculate and save final metrics for stage 1
    # _, _, _, video_preds, video_targets = validate(model, val_loader, criterion, config.DEVICE)
    # metrics = calculate_metrics(video_targets, video_preds)
    
    # print("\nStage 1 (Classifier Only) Results:")
    # for metric_name, metric_value in metrics.items():
    #     print(f"{metric_name}: {metric_value:.4f}")
    
    plot_metrics(train_losses_c, val_losses_c, train_accs_c, val_accs_c, video_accs_c, "classifier", config.OUTPUT_DIR)
    # plot_confusion_matrix(video_targets, video_preds, "classifier", config.OUTPUT_DIR)
    
    # STAGE 2: Fine-tune the entire model
    print("\n=== Training Stage 2: Fine-tuning ===")
    model.unfreeze_backbone()
    
    # Verify all parameters are now trainable
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Trainable parameters: {trainable_params} / {total_params} ({trainable_params/total_params:.2%})")
    
    # Define criterion and optimizer for fine-tuning
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), 
                           lr=config.LEARNING_RATE_FINETUNE, 
                           weight_decay=config.WEIGHT_DECAY)
    
    # Reset best accuracy for this stage
    best_video_acc = 0.0
    
    # Training loop
    train_losses_f = []
    val_losses_f = []
    train_accs_f = []
    val_accs_f = []
    video_accs_f = []
    
    for epoch in range(1, config.EPOCHS_FINETUNE + 1):
        print(f"\nEpoch {epoch}/{config.EPOCHS_FINETUNE}")
        
        # Train
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, config.DEVICE)
        
        # Validate
        val_loss, val_acc, video_acc, video_preds, video_targets = validate(model, val_loader, criterion, config.DEVICE)
        
        # Save metrics
        train_losses_f.append(train_loss)
        val_losses_f.append(val_loss)
        train_accs_f.append(train_acc)
        val_accs_f.append(val_acc)
        video_accs_f.append(video_acc)
        
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | Val Frame Acc: {val_acc:.4f} | Val Video Acc: {video_acc:.4f}")
        
        # Save checkpoint if it's the best model so far
        if video_acc > best_video_acc:
            best_video_acc = video_acc
            save_checkpoint(model, optimizer, epoch, val_loss, video_acc, "finetune_best", config.OUTPUT_DIR)
        
        # Save periodic checkpoint
        if epoch % config.CHECKPOINT_FREQ == 0:
            save_checkpoint(model, optimizer, epoch, val_loss, video_acc, f"finetune_epoch{epoch}", config.OUTPUT_DIR)
    
    # # Calculate and save final metrics for stage 2
    # _, _, _, video_preds, video_targets = validate(model, val_loader, criterion, config.DEVICE)
    # metrics = calculate_metrics(video_targets, video_preds)
    
    # print("\nStage 2 (Fine-tuning) Results:")
    # for metric_name, metric_value in metrics.items():
    #     print(f"{metric_name}: {metric_value:.4f}")
    
    plot_metrics(train_losses_f, val_losses_f, train_accs_f, val_accs_f, video_accs_f, "finetune", config.OUTPUT_DIR)
    #plot_confusion_matrix(video_targets, video_preds, "finetune", config.OUTPUT_DIR)
    
    # Save final model
    save_checkpoint(model, optimizer, config.EPOCHS_FINETUNE, val_loss, video_acc, "final", config.OUTPUT_DIR)
    
    # Save a torchscript version for easy deployment
    model.eval()
    scripted_model = torch.jit.script(model)
    scripted_model.save(os.path.join(config.OUTPUT_DIR, "deepfake_detector_scripted.pt"))
    
    print(f"\nTraining complete! All models saved to {config.OUTPUT_DIR}")


In [13]:
train_deepfake_detector()

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(



=== Training Stage 1: Classifier Only ===
Trainable parameters: 525058 / 21332010 (2.46%)

Epoch 1/5


Training:   0%|          | 0/2032 [00:00<?, ?it/s]

Validating:   0%|          | 0/511 [00:00<?, ?it/s]

Train Loss: 0.2897 | Train Acc: 0.9041
Val Loss: 0.2754 | Val Frame Acc: 0.9060 | Val Video Acc: 0.9061
Checkpoint saved at /kaggle/working/deepfake_detector_classifier_best_epoch_1.pth
Checkpoint saved at /kaggle/working/deepfake_detector_classifier_epoch1_epoch_1.pth

Epoch 2/5


...

Validating:   0%|          | 0/511 [00:00<?, ?it/s]

Train Loss: 0.0068 | Train Acc: 0.9977
Val Loss: 0.0461 | Val Frame Acc: 0.9868 | Val Video Acc: 0.9928
Checkpoint saved at /kaggle/working/deepfake_detector_finetune_epoch9_epoch_9.pth

Epoch 10/10


Training:   0%|          | 0/2032 [00:00<?, ?it/s]

Validating:   0%|          | 0/511 [00:00<?, ?it/s]

Train Loss: 0.0063 | Train Acc: 0.9978
Val Loss: 0.0625 | Val Frame Acc: 0.9863 | Val Video Acc: 0.9912
Checkpoint saved at /kaggle/working/deepfake_detector_finetune_epoch10_epoch_10.pth
Checkpoint saved at /kaggle/working/deepfake_detector_final_epoch_10.pth

Training complete! All models saved to /kaggle/working
